## Esktrakcja cech

In [59]:
import os.path
from pathlib import Path
import pandas as pd

path = '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation'

image_dir = Path(path)

filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.PNG')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

In [60]:
print(image_df)

                                               Filepath            Label
0     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight
1     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight
2     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight
3     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight
4     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight
...                                                 ...              ...
1763  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento
1764  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento
1765  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento
1766  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento
1767  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento

[1768 rows x 2 columns]


In [90]:
import os
import cv2
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from skimage.feature import canny
from skimage.exposure import rescale_intensity

def save_feature_image(image, op_path, method, file_suffix=''):
    method_path = os.path.join(op_path, method)
    if not os.path.exists(method_path):
        os.makedirs(method_path)
    save_path = os.path.join(method_path, f"{file_suffix}")
    if method != 'log' and 'canny':
        image_rescaled = (image - image.min()) / (image.max() - image.min()) * 255
        image = image_rescaled.astype('uint8')
    cv2.imwrite(save_path, image)

def lbp_feature(image, op_path, filename):
    lbp_image = local_binary_pattern(image, P=8, R=1, method='uniform')
    lbp_image = 255 - lbp_image
    save_feature_image(lbp_image, op_path, 'lbp', filename)

def canny_feature(image, op_path, filename):
    canny_img = canny(image, sigma=1.0)
    canny_image = (canny_img.astype(np.uint8) * 255).astype(np.uint8)
    save_feature_image(canny_image, op_path, 'canny', filename)

def hog_feature(image, op_path, filename):
    features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
    hog_image_rescaled = rescale_intensity(hog_image, in_range=(0, 10))
    save_feature_image(hog_image_rescaled, op_path, 'hog', filename)

def log_feature(image, op_path, filename):
    log_image = cv2.Laplacian(image, cv2.CV_16S, ksize=3)
    save_feature_image(log_image, op_path, 'log', filename)

# def save_histogram_features_as_image(features, op_path, filename):
#     # normalized_features = (features - np.min(features)) / (np.max(features) - np.min(features)) * 255
# 
#     # features_image = normalized_features.astype(np.uint8)
#     features_image = features.astype(np.uint8)
#     method_path = os.path.join(op_path, 'hist')
#     if not os.path.exists(method_path):
#         os.makedirs(method_path)
#     save_path = os.path.join(method_path, f"hist_{filename}")
#     cv2.imwrite(save_path, features_image)
# 
# def color_histogram(image):
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#     colors = ("r", "g", "b")
#     channels = cv2.split(image_rgb)
#     features = []
#     
#     for (channel, color) in zip(channels, colors):
#         hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
#         features.extend(hist.flatten())
# 
#     return features
# 
# def load_histogram_features_image(image_path):
#     features_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
# 
#     features = features_image.astype(np.float32) / 255.0 * (np.max(features_image) - np.min(features_image)) + np.min(features_image)
# 
#     return features

op_path_base = '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/feature_extraction/'

for row in image_df.itertuples(index=False):
    original_file = cv2.imread(row.Filepath)
    img = cv2.cvtColor(original_file, cv2.COLOR_BGR2GRAY)
    filename = os.path.basename(row.Filepath)
    op_path = os.path.join(op_path_base, row.Label)
    lbp_feature(img, op_path, filename)
    log_feature(img, op_path, filename)
    canny_feature(img, op_path, filename)
    hog_feature(img, op_path, filename)
    # histogram = color_histogram(img)
    # save_histogram_features_as_image(histogram, op_path, filename)

In [91]:
import os.path
from pathlib import Path
import pandas as pd

path = '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/feature_extraction/'

image_dir = Path(path)

filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.PNG')) + list(image_dir.glob(r'**/*.png'))

features = [os.path.split(os.path.dirname(x))[1] for x in filepaths]
labels = [os.path.split(os.path.split(os.path.dirname(x))[0])[1] for x in filepaths]

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
features = pd.Series(features, name='Feature')

image_df_features = pd.concat([filepaths, labels, features], axis=1)

In [92]:
print(image_df_features)

                                               Filepath            Label  \
0     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight   
1     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight   
2     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight   
3     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight   
4     /Users/kacperskurski/DataspellProjects/Soybean...  southern_blight   
...                                                 ...              ...   
7067  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento   
7068  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento   
7069  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento   
7070  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento   
7071  /Users/kacperskurski/DataspellProjects/Soybean...      crestamento   

     Feature  
0        lbp  
1        lbp  
2        lbp  
3        lbp  
4        lbp

## Trening modelu

In [94]:
import numpy as np
X = []

for index, row in image_df_features.iterrows():
    image_path = row['Filepath']
    labels = row['Label']
    features = row['Feature']
    if cv2.imread(image_path) is not None:
        if features != 'histogram':
            X.append(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY).flatten())
        # else:
        #     hist = load_histogram_features_image(image_path)
        #     print(hist)
        #     X.append(hist)
    
X = np.array(X)

In [95]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import cv2
import numpy as np

# X = np.array([cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY).flatten() for image_path in image_df_features.Filepath.values if cv2.imread(image_path) is not None])
y = np.array(image_df_features.Label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

model = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [96]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy}')

f_1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 score: {f_1}')

Model accuracy: 0.6657243816254417
F1 score: 0.6463918322844779


In [21]:
from sklearn.svm import SVC



svc_model = SVC(random_state=42)
svc_model.fit(X_train, y_train)

y_pred_svc = svc_model.predict(X_test)

accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f'Model accuracy: {accuracy_svc}')

f_1 = f1_score(y_test, y_pred_svc, average='weighted')
print(f'F1 score: {f_1}')

Model accuracy: 0.6600706713780918
F1 score: 0.6065467882299634


In [22]:
from sklearn.tree import DecisionTreeClassifier


dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f'Model accuracy: {accuracy_dt}')

f_1 = f1_score(y_test, y_pred_dt, average='weighted')
print(f'F1 score: {f_1}')

Model accuracy: 0.43886925795053006
F1 score: 0.441195452980562
